# Continuous Futures Contracts

Continuous contracts refer to methods that convert the prices associated with the individual futures expiration into longer term data by splicing data for different expirations together.

WARNING: In all cases of splicing and adjusting, it is important to avoid confusing these prices for representing
potential P&L or returns. We will return to this issue in a later unit.



In [1]:
import datetime
import sys
from itertools import cycle
from zoneinfo import ZoneInfo

import databento as db
import pandas as pd
import plotly.express as px

from finm37000 import (
    additive_splice,
    get_databento_api_key,
    multiplicative_splice,
    temp_env,
    us_business_day,
)

px.defaults.color_discrete_sequence = px.colors.qualitative.Set3
color_palette = cycle(px.defaults.color_discrete_sequence)

tz_chicago = ZoneInfo("America/Chicago")

sys.executable

'/Users/ericpatterson/.venv/finm37000-2025-09-17/bin/python'

In [2]:
with temp_env(DATABENTO_API_KEY=get_databento_api_key()):
    client = db.Historical()
start_of_this_year = (datetime.date(2025, 1, 1) + us_business_day).date()
yesterday = (datetime.datetime.now(tz=tz_chicago) - 2 * us_business_day).date()
cme = "GLBX.MDP3"

## Methods

### When To Splice

#### Roll by Days Until Expiration

Databento provides a roll _at expiration_ using the symbol `c` (for calendar).
By rolling at expiration, it does not cover cases like determining back month settlements
where the active contract rolls two days before expiration. Or does it? See September 17 below.

In [3]:
product = "CL"
roll_type = "c"
continuous_at_expiration = tuple(f"{product}.{roll_type}.{i}" for i in range(3))
continuous_at_expiration

('CL.c.0', 'CL.c.1', 'CL.c.2')

In [4]:
ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=continuous_at_expiration,
    stype_in="continuous",
    start=start_of_this_year,
    end=yesterday,
).to_df()

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_27560/3608279352.py:1: BentoWarning: The streaming request contained one or more days which have reduced quality: 2025-09-17 (degraded), 2025-09-24 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition
  ohlcv = client.timeseries.get_range(


In [5]:
raw_defs = client.timeseries.get_range(
    dataset=cme,
    schema="definition",
    symbols=f"{product}.FUT",
    stype_in="parent",
    start=start_of_this_year,
)

In [6]:
raw_defs.to_df()

,ts_event,rtype,publisher_id,instrument_id,raw_symbol,security_update_action,instrument_class,min_price_increment,display_factor,expiration,...,sub_fraction,underlying_product,maturity_month,maturity_day,maturity_week,user_defined_instrument,contract_multiplier_unit,flow_schedule_type,tick_rule,symbol
ts_recv,,,,,,,,,,,,,,,,,,,,,
2025-01-02 00:00:00+00:00,2024-12-29 12:06:29.885000+00:00,19,1,42286034,CL:C1 HO-CL H7,A,S,0.01,0.01,2027-02-22 19:30:00+00:00,...,255,16,3,255,255,N,127,127,255,CL:C1 HO-CL H7
2025-01-02 00:00:00+00:00,2024-12-29 12:06:28.385000+00:00,19,1,24567,CLQ5-CLN6,A,S,0.01,0.01,2025-07-22 18:30:00+00:00,...,255,16,8,255,255,N,127,127,255,CLQ5-CLN6
2025-01-02 00:00:00+00:00,2024-12-29 12:06:19.735683435+00:00,19,1,42003447,CLK35,A,F,0.01,0.01,2035-04-20 18:30:00+00:00,...,255,16,5,255,255,N,127,127,255,CLK35
2025-01-02 00:00:00+00:00,2024-12-29 12:06:28.985000+00:00,19,1,42019820,CLZ6-CLJ8,A,S,0.01,0.01,2026-11-20 19:30:00+00:00,...,255,16,12,255,255,N,127,127,255,CLZ6-CLJ8
2025-01-02 00:00:00+00:00,2024-12-29 12:06:30.135000+00:00,19,1,42068970,CLQ5-CLJ8,A,S,0.01,0.01,2025-07-22 18:30:00+00:00,...,255,16,8,255,255,N,127,127,255,CLQ5-CLJ8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-02 00:00:00+00:00,2024-12-29 12:06:26.235000+00:00,19,1,42068997,CLZ7-CLJ8,A,S,0.01,0.01,2027-11-19 19:30:00+00:00,...,255,16,12,255,255,N,127,127,255,CLZ7-CLJ8
2025-01-02 00:00:00+00:00,2024-12-29 12:06:28.035000+00:00,19,1,42339280,CLM7-CLX7,A,S,0.01,0.01,2027-05-20 18:30:00+00:00,...,255,16,6,255,255,N,127,127,255,CLM7-CLX7
2025-01-02 00:00:00+00:00,2024-12-29 12:06:22.685000+00:00,19,1,323547,CLM9,A,F,0.01,0.01,2029-05-22 18:30:00+00:00,...,255,16,6,255,255,N,127,127,255,CLM9


In [7]:
def_without_symbol = raw_defs.to_df()
assert (def_without_symbol["symbol"] == def_without_symbol["raw_symbol"]).all()
del def_without_symbol["symbol"]
del def_without_symbol["ts_event"]
ohlcv_exp = ohlcv.reset_index().merge(def_without_symbol, on="instrument_id")

In [8]:
fig = px.line(ohlcv_exp, x="ts_event", y="close", color="symbol")
fig.update_layout(title_text="Continuous contracts by expiration")
fig.show()

In [9]:
fig = px.line(ohlcv_exp, x="ts_event", y="close", color="raw_symbol")
fig.update_layout(title_text="Actual contract data composing continuous contracts")
fig.show()

In [10]:
continuous_symbol = continuous_at_expiration[0]
fig = px.line(
    ohlcv_exp[ohlcv_exp["symbol"] == continuous_symbol],
    x="ts_event",
    y="close",
    color="raw_symbol",
)
fig.update_layout(
    title_text="Actual contract data composing the first continuous contracts"
)
fig.show()

#### Roll by Volume

Similarly, the symbol `v` can be used to request continuous contracts by volume. The contract changes the day _after_
the contract ordering by volume changes.

In [11]:
roll_type = "v"
continuous_by_volume = tuple(f"{product}.{roll_type}.{i}" for i in range(3))
continuous_by_volume

('CL.v.0', 'CL.v.1', 'CL.v.2')

In [12]:
ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=continuous_by_volume,
    stype_in="continuous",
    start=start_of_this_year,
    end=yesterday,
).to_df()

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_27560/190630183.py:1: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2025-09-17 (degraded), 2025-09-24 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [13]:
ohlcv_vol = ohlcv.reset_index().merge(def_without_symbol, on="instrument_id")

In [14]:
continuous_symbol = continuous_by_volume[0]
fig = px.line(
    ohlcv_vol[ohlcv_vol["symbol"] == continuous_symbol],
    x="ts_event",
    y="close",
    color="raw_symbol",
)
fig.update_layout(
    title_text="Actual contract data composing the first continuous contract by volume"
)
fig.show()

In [15]:
continuous_symbol = continuous_by_volume[1]
fig = px.line(
    ohlcv_vol[ohlcv_vol["symbol"] == continuous_symbol],
    x="ts_event",
    y="close",
    color="raw_symbol",
)
fig.update_layout(
    title_text="Actual contract data composing the second continuous contract by volume"
)
fig.show()

Can you explain why it looks like this?

In [16]:
continuous_symbol = continuous_by_volume[1]
fig = px.scatter(
    ohlcv_vol[ohlcv_vol["symbol"] == continuous_symbol],
    x="ts_event",
    y="close",
    color="raw_symbol",
)
fig.update_layout(
    title_text="Actual contract data composing the second continuous contract by volume"
)
fig.show()

#### Roll by Open Interest

Similar to the volume logic, we can switch contracts when the open interest ordering changes. Databento
supports this with the `n` `roll_type`.



In [17]:
roll_type = "n"
continuous_by_oi = tuple(f"{product}.{roll_type}.{i}" for i in range(3))
continuous_by_oi

('CL.n.0', 'CL.n.1', 'CL.n.2')

In [18]:
ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=continuous_by_oi,
    stype_in="continuous",
    start=start_of_this_year,
    end=yesterday,
).to_df()

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_27560/1360519876.py:1: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2025-09-17 (degraded), 2025-09-24 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [19]:
ohlcv_oi = ohlcv.reset_index().merge(def_without_symbol, on="instrument_id")
ohlcv_oi["symbol"]

0      CL.n.2
1      CL.n.0
2      CL.n.1
3      CL.n.2
4      CL.n.0
        ...  
766    CL.n.2
767    CL.n.0
768    CL.n.1
769    CL.n.2
770    CL.n.0
Name: symbol, Length: 771, dtype: object

In [ ]:
continuous_symbol = continuous_by_oi[0]
fig = px.line(
    ohlcv_oi[ohlcv_oi["symbol"] == continuous_symbol],
    x="ts_event",
    y="close",
    color="raw_symbol",
)
fig.update_layout(
    title_text="Contracts composing the first continuous contract by open interest",
)
fig.show()

In [ ]:
continuous_symbol = continuous_by_oi[1]
fig = px.line(
    ohlcv_oi[ohlcv_oi["symbol"] == continuous_symbol],
    x="ts_event",
    y="close",
    color="raw_symbol",
)
fig.update_layout(
    title_text="Contracts composing the second continuous contract by open interest",
)
fig.show()

In [ ]:
continuous_symbol = continuous_by_oi[1]
fig = px.scatter(
    ohlcv_oi[ohlcv_oi["symbol"] == continuous_symbol],
    x="ts_event",
    y="close",
    color="raw_symbol",
)
fig.update_layout(
    title_text="Contracts composing the second continuous contract by open interest",
)
fig.show()

### Checking the roll specification

You can comb through the data frame to see where the contracts switch. Databento provides a convenient
resolution specification to summarize what it is doing, although it only outputs by `instrument_id`
as this time.

In [23]:
client.symbology.resolve(
    dataset=cme,
    symbols=continuous_by_oi,
    stype_in="continuous",
    stype_out="instrument_id",
    start_date=yesterday - 20 * us_business_day,
    end_date=yesterday,
)

{'result': {'CL.n.0': [{'d0': '2025-10-01',
    'd1': '2025-10-30',
    's': '651434'}],
  'CL.n.1': [{'d0': '2025-10-01', 'd1': '2025-10-10', 's': '432669'},
   {'d0': '2025-10-10', 'd1': '2025-10-30', 's': '240384'}],
  'CL.n.2': [{'d0': '2025-10-01', 'd1': '2025-10-10', 's': '240384'},
   {'d0': '2025-10-10', 'd1': '2025-10-15', 's': '432669'},
   {'d0': '2025-10-15', 'd1': '2025-10-30', 's': '447685'}]},
 'symbols': ['CL.n.0', 'CL.n.1', 'CL.n.2'],
 'stype_in': 'continuous',
 'stype_out': 'instrument_id',
 'start_date': '2025-10-01',
 'end_date': '2025-10-30',
 'partial': [],
 'not_found': [],
 'message': 'OK',
 'status': 0}

In [24]:
client.symbology.resolve(
    dataset=cme,
    symbols=continuous_at_expiration,
    stype_in="continuous",
    stype_out="instrument_id",
    start_date=yesterday - 20 * us_business_day,
    end_date=yesterday,
)

{'result': {'CL.c.0': [{'d0': '2025-10-01', 'd1': '2025-10-22', 's': '432669'},
   {'d0': '2025-10-22', 'd1': '2025-10-30', 's': '651434'}],
  'CL.c.1': [{'d0': '2025-10-01', 'd1': '2025-10-22', 's': '651434'},
   {'d0': '2025-10-22', 'd1': '2025-10-30', 's': '240384'}],
  'CL.c.2': [{'d0': '2025-10-01', 'd1': '2025-10-22', 's': '240384'},
   {'d0': '2025-10-22', 'd1': '2025-10-30', 's': '383438'}]},
 'symbols': ['CL.c.0', 'CL.c.1', 'CL.c.2'],
 'stype_in': 'continuous',
 'stype_out': 'instrument_id',
 'start_date': '2025-10-01',
 'end_date': '2025-10-30',
 'partial': [],
 'not_found': [],
 'message': 'OK',
 'status': 0}

### How To Splice

Many intraday strategies and models do not require any adjustments. The roll calendar
may provide a guide to how the contracts traded by an intraday strategy evolve by date,
so it is still relevant, and we will have more to say about that workflow in the next unit.

Splicing together prices for historical display and analysis typically provide adjustments.
There are many sources and references that provide various adjusted series. It is not always
clear what methods they follow, and there are many potential pitfalls that can arise.
When incorporating these into an analysis, it is important to determine what methods have
been used to maintain reproducible result free of look-forward bias.


#### Unadjusted Splice

Unadjusted continuous contracts make no attempt to correct for expiration and price differences. This is the
approach used by Databento, for example. That is, all the preceding examples are unadjusted splices.

The biggest plus of unadjusted splices is that the prices all represent real prices of real contracts.
The biggest drawback of unadjusted splices is that the prices do not represent the price of a single contract,
so calculating returns or P&L naively will be incorrect. Additional assumptions need to be made to exit an expiring
contract and enter the next contraction in order to generate returns or P&L.


#### Additive/Panama Canal

A common method of adjustment is to add the difference between exiting the expiring contract and entering the
next contract on a specific roll date. At each roll date, the adjustments compound (i.e., the second roll adjustment
needs to account for the first roll adjustment, etc.). It is fairly common to use a closing value to make
this adjustment, but closing values are not standardized, so there is room for discrepancies among vendors.
And alternatively, values other than the close can be used to make the adjustment.

An important consideration is whether the adjustments compound in a forward manner or a backward manner.
Due to my own biases, I favor the forward adjustment, but the backward adjustment is far more popular and
will be discussed more below. Here is a forward adjusted example.

In [25]:
long_history_start = (datetime.date(2018, 1, 1) + us_business_day).date()
extended_defs = client.timeseries.get_range(
    dataset=cme,
    schema="definition",
    symbols=f"{product}.FUT",
    stype_in="parent",
    start=long_history_start,
)

In [26]:
roll_spec = client.symbology.resolve(
    dataset=cme,
    symbols=continuous_at_expiration,
    stype_in="continuous",
    stype_out="instrument_id",
    start_date=long_history_start,
    end_date=yesterday,
)
roll_spec

{'result': {'CL.c.0': [{'d0': '2018-01-02', 'd1': '2018-01-23', 's': '253894'},
   {'d0': '2018-01-23', 'd1': '2018-02-21', 's': '669714'},
   {'d0': '2018-02-21', 'd1': '2018-03-21', 's': '477338'},
   {'d0': '2018-03-21', 'd1': '2018-04-22', 's': '545967'},
   {'d0': '2018-04-22', 'd1': '2018-05-23', 's': '45928'},
   {'d0': '2018-05-23', 'd1': '2018-06-21', 's': '353594'},
   {'d0': '2018-06-21', 'd1': '2018-07-22', 's': '253896'},
   {'d0': '2018-07-22', 'd1': '2018-08-22', 's': '738344'},
   {'d0': '2018-08-22', 'd1': '2018-09-21', 's': '820474'},
   {'d0': '2018-09-21', 'd1': '2018-10-23', 's': '305220'},
   {'d0': '2018-10-23', 'd1': '2018-11-20', 's': '60391'},
   {'d0': '2018-11-20', 'd1': '2018-12-20', 's': '110955'},
   {'d0': '2018-12-20', 'd1': '2019-01-23', 's': '303281'},
   {'d0': '2019-01-23', 'd1': '2019-02-21', 's': '79323'},
   {'d0': '2019-02-21', 'd1': '2019-03-21', 's': '97456'},
   {'d0': '2019-03-21', 'd1': '2019-04-23', 's': '694785'},
   {'d0': '2019-04-23', 

In [27]:
required_ids = [
    spec["s"]
    for continuous_contract, spec_list in roll_spec["result"].items()
    for spec in spec_list
]
ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=required_ids,
    stype_in="instrument_id",
    start=long_history_start,
    end=yesterday,
).to_df()

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_27560/51314230.py:6: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2018-10-21 (degraded), 2019-01-15 (degraded), 2019-02-22 (degraded)... See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [28]:
unadjusted_raw = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=continuous_at_expiration[0],
    stype_in="continuous",
    start=long_history_start,
    end=yesterday,
)

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_27560/3839711639.py:1: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2018-10-21 (degraded), 2019-01-15 (degraded), 2019-02-22 (degraded)... See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [29]:
continuous_symbol = "CL.c.0"
additive_adjusted = additive_splice(
    roll_spec["result"][continuous_symbol],
    ohlcv.reset_index(),
    date_col="ts_event",
    adjust_by="close",
)
unadjusted = unadjusted_raw.to_df().reset_index()
unadjusted["adjustment"] = "unadjusted"

In [30]:
additive_adjusted["adjustment"] = "additive"
plot_df = pd.concat([unadjusted, additive_adjusted], ignore_index=True)
fig = px.line(plot_df, x="ts_event", y="close", color="adjustment")
fig.update_layout(
    title_text=f"Additive adjustment on expiration date, {continuous_symbol}"
)
fig.show()

#### Multiplicative/Ratio

Another common adjustment method is multiplying by the ratio of the contract prices at the roll date.
This preserves individual contract returns across the roll date.

As with the additive adjustment, this can be done compounding forward or backward, and I present the forward
adjustment here.

In [31]:
multiplicative_adjusted = multiplicative_splice(
    roll_spec["result"][continuous_symbol],
    ohlcv.reset_index(),
    date_col="ts_event",
    adjust_by="close",
)

In [ ]:
multiplicative_adjusted["adjustment"] = "multiplicative"
plot_df = pd.concat(
    [unadjusted, additive_adjusted, multiplicative_adjusted], ignore_index=True
)
fig = px.line(plot_df, x="ts_event", y="close", color="adjustment")
fig.update_layout(title_text=f"Adjustments on expiration date, {continuous_symbol}")
fig.show()

What could go wrong with rolling on expiration?

If choose a different roll date rather than expiration, the adjusted continuous prices will change.
For example, here is the volume-based roll.

In [33]:
roll_spec = client.symbology.resolve(
    dataset=cme,
    symbols=continuous_by_volume,
    stype_in="continuous",
    stype_out="instrument_id",
    start_date=long_history_start,
    end_date=yesterday,
)

In [34]:
required_ids = [
    spec["s"]
    for continuous_contract, spec_list in roll_spec["result"].items()
    for spec in spec_list
]
ohlcv = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=required_ids,
    stype_in="instrument_id",
    start=long_history_start,
    end=yesterday,
).to_df()

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_27560/51314230.py:6: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2018-10-21 (degraded), 2019-01-15 (degraded), 2019-02-22 (degraded)... See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [35]:
continuous_symbol = "CL.v.0"
additive_adjusted = additive_splice(
    roll_spec["result"][continuous_symbol],
    ohlcv.reset_index(),
    date_col="ts_event",
    adjust_by="close",
)
multiplicative_adjusted = multiplicative_splice(
    roll_spec["result"][continuous_symbol],
    ohlcv.reset_index(),
    date_col="ts_event",
    adjust_by="close",
)

In [36]:
unadjusted_raw = client.timeseries.get_range(
    dataset=cme,
    schema="ohlcv-1d",
    symbols=continuous_symbol,
    stype_in="continuous",
    start=long_history_start,
    end=yesterday,
)
unadjusted = unadjusted_raw.to_df().reset_index()
unadjusted["adjustment"] = "unadjusted"

/var/folders/8y/2f_yzjcd2tz12vswl92n4vpw0000gp/T/ipykernel_27560/3589730698.py:1: BentoWarning:

The streaming request contained one or more days which have reduced quality: 2018-10-21 (degraded), 2019-01-15 (degraded), 2019-02-22 (degraded)... See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition



In [ ]:
additive_adjusted["adjustment"] = "additive"
multiplicative_adjusted["adjustment"] = "multiplicative"
plot_df = pd.concat(
    [unadjusted, additive_adjusted, multiplicative_adjusted], ignore_index=True
)
fig = px.line(plot_df, x="ts_event", y="close", color="adjustment")
fig.update_layout(title_text=f"Adjustments with volume roll, {continuous_symbol}")
fig.show()

#### Multiday Adjustments

There are several methods for extending adjustments over several days. For example, a common rollover technique
is to define a multiday window over which to roll the price, say 5 days, with the percent of the old contract
decreasing as the new contract increases, say 100%, 80%, 60%, 40%, 20%, 0%. This may align with a trading
schedule for rolling over a large position over multiple days, allowing the adjusted price to approximate
the position value.

#### Constant Maturity Roll

Futures expirations can be averaged rather than spliced, extending the idea of multiday adjustment.
This is a useful data processing step to remove the time-to-expiration dependence of futures prices.

(What other ways can we try to remove this time-to-expiration dependence?)

Suppose you want a constant maturity price with $m$ days until maturity at time $t$.
Look ahead to $t+m$ and find the previous and next expirations around that date, $T_p$ and $T_n$.
We can weight their prices $F_{p,t}$ and $F_{n,t}$ to produce an $m$-day constant maturity price:
$$
CM_{m,t} = \frac{T_n-(t+m)}{T_n-T_p} F_{p,t} + \frac{(t+m)-T_p}{T_n-T_p} F_{n,t}
$$
or letting $d=T_p-t$ be the number of days until $T_p$ and $s=T_n-T_p$ the days between expirations
$$
CM_{m,t} = \frac{s+d-m}{s} F_{p,t} + \frac{m-d}{s} F_{n,t}
$$

#### Back-adjusted Splicing

Back-adjusted splicing (multiplicative and additive) is often used because today's adjusted values match
the unadjusted values, so they provide a clear picture of the current market environment.
They are the dominant paradigm when plotting and displaying continuous contract futures, but they
are more difficult to work with correctly to analyze historical data because they introduce lookforward bias.

For example, here is a back-adjusted ratio history of Lean Hogs (https://quantpedia.com/continuous-futures-contracts-methodology-for-backtesting/)


![backadjusted_lean_hogs.png](img/backadjusted_lean_hogs.png)

This is the unadjusted history.

![unadjusted_lean_hogs.png](img/unadjusted_lean_hogs.png)